In [ ]:
base_pkl = '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M10_Sk0/epistemic_uncertainty_data_collection/data.dset_type-2ms/stats_training_disentangle_2406_D25-M3-S0-L8_4.pkl'

In [ ]:
base_noise_level = base_pkl.split('/')[-2].replace('data.dset_type-', '')
assert base_noise_level in base_pkl, f'{base_noise_level} not in {base_pkl}'
print(base_noise_level)
different_noise_levels = {
    '2ms': base_pkl.replace(base_noise_level, '2ms'),
    '3ms': base_pkl.replace(base_noise_level, '3ms'),
    '5ms': base_pkl.replace(base_noise_level, '5ms'),
    '20ms': base_pkl.replace(base_noise_level, '20ms'),
    '500ms': base_pkl.replace(base_noise_level, '500ms'),
    # 'Ch': '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M10_Sk0/epistemic_uncertainty_data_collection/data.channel_idx_list-[1, 2, 0, 14]/stats_training_disentangle_2406_D25-M3-S0-L8_4.pkl'
}

In [ ]:
import pickle
data_dict = {}
for k, v in different_noise_levels.items():
    with open(v, 'rb') as f:
        data_dict[k] = pickle.load(f)

In [ ]:
data_dict['2ms'].keys()

In [ ]:
ch_idx = 0
img_idx = 0
data_dict['2ms']['pred'][ch_idx][img_idx].shape

### Compute Epistemic Uncertainty

In [ ]:
for conf_key in data_dict.keys():
    data_dict[conf_key]['std'] ={}
    for ch_idx in data_dict[conf_key]['pred'].keys():
        data_dict[conf_key]['std'][ch_idx] = []
        for img_idx in range(len(data_dict[conf_key]['pred'][ch_idx])):
            img = data_dict[conf_key]['pred'][ch_idx][img_idx]
            std = img.std(axis=0)
            data_dict[conf_key]['std'][ch_idx].append(std)


In [ ]:
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
from tqdm import tqdm
save_to_file = True

_,ax = plt.subplots(figsize=(15,4),ncols=3)
for conf_key in tqdm(data_dict.keys()):
    for ch_idx in range(3):
        std_values = data_dict[conf_key]['std'][ch_idx]
        # one value per patch
        std_values = [np.mean(x[16:-16,16:-16]) for x in std_values]
        # add linewidth=10
        sns.kdeplot(std_values,ax=ax[ch_idx], log_scale=True, label=conf_key, 
                    linewidth=3 if conf_key == base_noise_level else 1,
                    color = 'green' if conf_key == base_noise_level else None)
    #

for ch_idx in range(3):
    ax[ch_idx].grid()
    # grid background color to gray
    ax[ch_idx].set_facecolor('lightgrey')

# ax[-1].legend()
# set legend title 
ax[-1].legend(title='Eval Data')

if save_to_file:
    model_id = base_pkl.split('/')[-1].replace('.pkl', '').replace('stats_training_disentangle_','')
    plot_fpath = os.path.join('/group/jug/ashesh/naturemethods/epistemic_uncertainty', f'noise_variation_{base_noise_level}_{model_id}.png')
    print(plot_fpath)
    plt.savefig(plot_fpath, dpi=200)


In [ ]:
print(plt.rcParams['axes.prop_cycle'].by_key()['color'])